In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading CSV dataset with ANSI encoding as it includes German Tokens

df = pd.read_csv(r"dataset/accidents.csv", encoding = "ANSI")
df.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunf?le,insgesamt,2021,202101,NaN,28.0,NaN,NaN,NaN
1,Alkoholunf?le,insgesamt,2021,202102,NaN,40.0,NaN,NaN,NaN
2,Alkoholunf?le,insgesamt,2021,202103,NaN,27.0,NaN,NaN,NaN
3,Alkoholunf?le,insgesamt,2021,202104,NaN,26.0,NaN,NaN,NaN
4,Alkoholunf?le,insgesamt,2021,202105,NaN,40.0,NaN,NaN,NaN


In [3]:
# Recognizing first 5 columns for further analysis

df = df.iloc[:, :5]
df = df.dropna()
df.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
12,Alkoholunf?le,insgesamt,2020,Summe,430.0
13,Alkoholunf?le,insgesamt,2020,202001,28.0
14,Alkoholunf?le,insgesamt,2020,202002,40.0
15,Alkoholunf?le,insgesamt,2020,202003,27.0
16,Alkoholunf?le,insgesamt,2020,202004,26.0
